In [13]:
import sys
sys.path.append('.')
from DDN import DDN

import numpy as np
from sklearn.linear_model import Lasso

class DDN_Lasso(DDN):
    def lasso(self, y, X, lambda1, strongrule=True, tol=1e-6):
        # select feature index
        idx = np.arange(X.shape[1])
    
        # strong rule
        def strongRule(X, y, lambda1):
            lambdas = np.matmul(X.T, y)
            lambdamax = max(lambdas)
            idx = np.array([i for i in range(X.shape[1]) if lambdas[i] >= 2 * lambda1 - lambdamax])
            drop = X.shape[1] - len(idx)
            if drop:
                print(f"Lasso Strong Rule: {drop} predictors dropped!")

            return idx

        if strongrule:
            idx = strongRule(X, y, lambda1)
            X = X[:, idx]

        # calculate beta with sklearn lasso
        clf = Lasso(alpha=lambda1, max_iter=1000, tol=tol, fit_intercept=False, warm_start=True)
        clf.coef_ = np.zeros((X.shape[1], ))
        clf.fit(X, y)

        beta = np.zeros((len(idx), ))
        beta[idx] = clf.coef_

        return beta
        
    
    def generateDifferentialNetwork(self, case_data, control_data, genename, lambda1=0.30, lambda2=0.00):
        # check lambda2
        assert(lambda2 == 0.0)
        
        # feature size (gene size)
        p = control_data.shape[1]
        
        # start calculations
        diffedges = {}
        for gene in range(p):
            # choose one gene as target
            y1 = control_data[:, gene]
            y2 = case_data[:, gene]
            
            # choose other genes as feature
            idx = [i for i in range(p) if i != gene]
            X1 = control_data[:, idx]
            X2 = case_data[:, idx]
            
            # perform bcd algorithm
            beta1 = self.lasso(y1, X1, lambda1)
            beta2 = self.lasso(y2, X2, lambda1)
            
            # reindex the features
            beta1 = list(beta1[0:gene]) + [0] + list(beta1[gene:])
            beta1 = np.array(beta1)
            beta2 = list(beta2[0:gene]) + [0] + list(beta2[gene:])
            beta2 = np.array(beta2)
            
            # construct neighbours under two conditions
            condition1 = [genename[i] for i in range(p) if beta1[i] != 0 and beta2[i] == 0]
            condition2 = [genename[i] for i in range(p) if beta2[i] != 0 and beta1[i] == 0]
            weight1 = [beta1[i] for i in range(p) if beta1[i] != 0 and beta2[i] == 0]
            weight2 = [beta2[i] for i in range(p) if beta2[i] != 0 and beta1[i] == 0]
            
            # update results
            for neighbors, weights, condition in zip([condition1, condition2], [weight1, weight2], ['condition1', 'condition2']):
                for neighbor, weight in zip(neighbors, weights):
                    tuple_diffedge = (min(genename[gene], neighbor), max(genename[gene], neighbor), condition)
                    diffedges.setdefault(tuple_diffedge, 0.0)
                    diffedges[tuple_diffedge] += weight
        
        diffedges = sorted([k + tuple([v]) for k, v in diffedges.items()])
        
        return diffedges
    
    def DDNPipline(self, case_data_file, control_data_file, gene_name_file, output_file='', lambda1=0.30, lambda2=0.00):
        # import case data
        casedata = self.readGeneData(case_data_file)
        
        # import control data
        controldata = self.readGeneData(control_data_file)
        
        # import gene name
        genename = self.readGeneName(gene_name_file)
        
        # feature size must be equivalent
        assert(casedata.shape[1] == controldata.shape[1])
        
        # feature standardization
        case_standard = self.standardizeGeneData(casedata)
        control_standard = self.standardizeGeneData(controldata)
        
        # generate differential network
        diffedges = self.generateDifferentialNetwork(case_standard, control_standard, genename, lambda1, lambda2)
        
        # print differential network
        self.printDifferentialNetwork(diffedges, output_file)
        
        return diffedges
    

In [14]:
%%time

ddn = DDN_Lasso()
neighbors = ddn.DDNPipline(case_data_file='dataset_syntren_case.txt', \
                           control_data_file='dataset_syntren_control.txt', \
                           gene_name_file='dataset_syntren_genename.txt', \
                           #output_file='', \
                           lambda1=0.8, lambda2=0.00)
neighbors1 = neighbors
len(neighbors)

DNN package
ACE2,CDC10,condition2,0.08391972587124115
ACE2,CTS1,condition1,0.06585313751168127
ACT1,HTB1,condition1,0.14392016707751254
ACT1,SPT16,condition2,0.1295833102858191
CDC10,SWI4,condition2,0.15362602604168146
CDC11,CTS1,condition2,0.14377802612543789
CDC11,FLO1,condition1,0.0823353993648766
CDC11,HO,condition2,0.015081125388273333
CDC11,SWI4,condition1,0.14713310404153704
CDC11,SWI4_SWI6,condition2,0.11198625782249609
CLB5,MBP1_SWI6,condition2,0.14138136170883853
CLB5,PHO2,condition1,0.1191672116566562
CLB6,PHO2,condition1,0.0823320969282986
CTS1,SWI4_SWI6,condition2,0.022766199420347563
CTS1,TRP4,condition2,0.025166955655331835
FLO1,FLO10,condition2,0.041793920313868124
FLO1,PHO2,condition2,0.10637576147052474
FLO1,TRP4,condition1,0.045782617977691016
FLO10,PHO2,condition2,0.3130780652548436
FLO10,TRP4,condition1,0.1313912516150843
HTB1,SPT16,condition1,0.014936311562368161
LEU2,SWI4_SWI6,condition1,0.04057405899402199
MBP1_SWI6,TRP4,condition1,0.10409657867069638
PHO2,TRP4,

25